# Análisis y traducción de tweets de texto común a ASCII

Notebook generadora del dataset para utilizar en el análisis. Toma la base de datos, remueve las palabras y signos a ignorar, traduce los emojis a ASCII y guarda el dataset final.

In [333]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from TweetAnalysis import hermes

#### Cargo la base de datos clasificada y genero las variables de importancia

In [334]:
TW = pd.read_csv('base_corr.csv')

Estos son los tweets no clasificados

In [335]:
DATA = pd.read_csv('tweets.csv',sep=';')

MSJ contiene al tweet, Tag si es meteorológico o no, Num el id del tweet, Who el nombre de quien los clasificó y User el id de quien twiteó (eso lo tuvimos que sacar de la base de datos original).

In [336]:
MSJ = TW['Twit']
TAG = TW['Tag']
NUM = TW['Num']
WHO = TW['Nombre']
USER = DATA['user_id'][np.asarray(NUM)]

In [337]:
FINAL = pd.DataFrame(list(zip(MSJ, TAG, NUM, USER, WHO)), columns = ['MSJ', 'TAG', 'NUN', 'USER', 'WHO'])

## La función Hermes: normaliza los tweets y borra el spam meteorológico

In [340]:
normalizado = hermes(FINAL, 'MSJ', 'USER', 5)
normalizado

,MSJ,TAG,NUN,USER,WHO,msj_norm
0,Esta lloviendo,1,NaN,NaN,NaN,esta lloviendo
1,quiero ir al centro a la mañana pero seguro ll...,0,41082.0,1.027416e+18,NaN,quiero ir centro mañana pero seguro llueve
2,"@paloma_CAI7 Si, llueve y mucho !! 😒",1,86721.0,9.704968e+17,NaN,si llueve mucho \U0001f612
3,@rozalazar9 Si llueve les hago 🤗,0,107171.0,1.708397e+09,NaN,si llueve hago \U0001f917
4,Hasta cuándo le voy a seguir teniendo miedo a ...,0,57648.0,8.477091e+17,NaN,hasta cuando voy seguir teniendo miedo torment...
6,"@Eze2450 Sea la estación que sea , no hay plat...",1,67700.0,1.912568e+08,NaN,sea estacion sea no hay plata ahora llueve...
7,Llueve y no tengo Internet...me espera tremend...,1,68738.0,2.907214e+09,NaN,llueve no tengo internet espera tremenda tarde
8,Que linda noche de lluvia 💖,1,41774.0,2.266866e+09,NaN,que linda noche lluvia \U0001f496
9,"Miércoles de mucho calor, previo a las lluvias...",1,42346.0,1.737797e+08,NaN,miercoles mucho calor previo lluvias
10,Me gusta cuando llueve porque parece que se es...,0,17694.0,2.560815e+09,NaN,me gusta cuando llueve porque parece estan fri...


## Corrector ortográfico

In [208]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('listado-general.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

#### División en datasets de  _train_ y _test_

In [ ]:
size = 0.8

## Toma los tweets, los asocia con su tag y luego los divide en cada una de las palabras que los componen
## zip() itera tuplas

train = [(i.split(),j) for i,j in zip(list(msj)[1:int(len(msj)*0.8)],list(TAG)[1:int(len(msj)*0.8)])]

test = [(i.split(),j) for i,j in zip(list(msj)[int(len(msj)*0.8)+1:len(msj)],list(TAG)[int(len(msj)*0.8)+1:len(msj)])]

## Guardo los labels, (en este caso 1 y 0) en un diccionario
label_to_ix = { 1: 0, 0: 1 } 

# 1 = Meteorológico
# 0 = No meteorológico



#### Generación del diccionario con el dataset completo

In [ ]:
word_to_ix = {}

# El diccionario tiene que tener todas las palabras posibles
datos = train + test

## Cada palabra que encuentra la guarda en un diccionario
for sent, _ in datos:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2